# NCPT Paperspace One-click (v0.3)

### Last updated: 25 Dec 2023

Check for updates [here](https://github.com/NoCrypt/paperspace-ncpt)

**!! ONLY RUN IN JUPYTERLAB !!**

![](https://i.ibb.co/M9ZH6qf/image.png)




To hide the code, press on this blue bar on the left of the cell

![](https://i.ibb.co/gFqKVV6/chrome-23-12-25-195649.png)

---


#### 1. Configurations

In [ ]:
import requests, time, json, os
from urllib.parse import urlparse, parse_qs, unquote
from ipywidgets import Checkbox, Text, Button, HBox, ToggleButton, Label, Box, Layout, Accordion, VBox
from IPython.display import display, HTML
from pathlib import Path
from subprocess import check_output
import glob


# Folder size method 
def get_directory_size(directory_pattern):
    try:
        if '*' in directory_pattern:
            directories = glob.glob(directory_pattern)
        else:
            directories = [directory_pattern] if os.path.isdir(directory_pattern) else []
        if not directories:
            return "0B"
        total_size = 0
        for directory in directories:
            output = check_output(['du', '-s', directory], text=True)
            size = int(output.split()[0])  # Size in blocks
            total_size += size * 1024  # Convert to bytes (assuming block size is 1024 bytes)

        # Convert bytes to a human-readable format
        for unit in ['B', 'KB', 'MB', 'GB', 'TB', 'PB']:
            if total_size < 1024:
                return f"{total_size:.2f}{unit}"
            total_size /= 1024

        return f"{total_size:.2f}PB"
    except Exception as e:
        print("An error occurred:", e)
        return "0B"

 
# Mapping resources folders 
paths = {
    "models": "models/Stable-diffusion",
    "loras": "models/Lora",
    "vaes": "models/VAE",
    "embeddings": "embeddings",
    "extensions":  "extensions",  
}

class Params:
    # default parameters
    file_path=""
    controls =[]
    accordions ={}
    links={
        "models": ["https://huggingface.co/NoCrypt/expermental_models/resolve/main/shux7.safetensors"],
        "vaes":["https://huggingface.co/NoCrypt/resources/resolve/main/VAE/any.vae.safetensors"],
        "loras": [],
        "extensions": [],
        "embeddings": [],
        "ui": {"params": {},"enabled": ["https://huggingface.co/NoCrypt/expermental_models/resolve/main/shux7.safetensors","https://huggingface.co/NoCrypt/resources/resolve/main/VAE/any.vae.safetensors"],"keep": [],"names": {},"tabs": []}}


    def __init__(self, control_list):
        self.controls = control_list

    # load parameters from file otherwise it creates a default one 
    def load(self, file = "links.json"):
        self.file_path = file

        if not os.path.exists(self.file_path):
            self.save()
        else :
            with open(self.file_path, 'r') as file:
                self.links = json.load(file)

        for ctrl in self.controls:
            if ctrl.tooltip in self.links["ui"]["params"] :
                ctrl.value = self.links["ui"]["params"][ctrl.tooltip]

    # update parameters from ui 
    def update(self):
        for ctrl in self.controls:
            self.links["ui"]["params"][ctrl.tooltip] = ctrl.value
            
    # save parameters to file           
    def save(self):
        self.update()
        with open(self.file_path, 'w') as file:
            json.dump(self.links, file, indent=4)

    # get the status of the ressource enabled/cached
    def get_link_status(self, url):
        enabled = url in self.links["ui"]["enabled"]
        keep = url in self.links["ui"]["keep"]
        return enabled, keep

    # get the ressource name from parameters
    def get_link_name(self , url):
        if 'names' in self.links["ui"] and url in self.links["ui"]['names']:
            return self.links["ui"]['names'][url]
        else:
            return url
            
    # update the ressource status
    def update_link_status(self, url, to_enable, to_keep):
        enabled, keep = self.get_link_status(url)
        if to_enable :
            if not enabled :
                self.links["ui"]["enabled"].append(url)
        else :
            if enabled :
                self.links["ui"]["enabled"].remove(url)
                
        if to_keep :
            if not keep :
                self.links["ui"]["keep"].append(url)
        else :
            if keep :
                self.links["ui"]["keep"].remove(url)
        self.save()

    # adds a new ressource
    def add_link(self, url, category):
        def is_valid_url(url):
            parsed = urlparse(url)
            return bool(parsed.scheme) and bool(parsed.netloc)
            
        url=url.strip()
        if is_valid_url(url) and  url not in self.links[category] :
            self.links[category].append(url)
            self.links["ui"]["enabled"].append(url)
            self.get_accordion(category)
        self.save()
        
    # removes the specified ressource
    def remove_link(self, cat, elem):
        if elem in self.links[cat] :
            self.links[cat].remove(elem)
        self.get_accordion(cat) 
        self.save()


    # create or update the ressource manager 
    def get_accordion(self, category, refresh=False):
        is_new=False
        if category not in self.accordions:
            is_new=True
            self.accordions[category] = Accordion(layout=Layout(width='52%'))
            def on_accordion_change(change):
                if "tabs" not in self.links["ui"] :
                    self.links["ui"]["tabs"]=[]
                
                if change['name'] == 'selected_index' and change['new'] is not None:
                    if category not in self.links["ui"]["tabs"]:
                        self.links["ui"]["tabs"].append(category)
                else :
                    if category in self.links["ui"]["tabs"]:
                        self.links["ui"]["tabs"].remove(category)
            self.accordions[category].observe(on_accordion_change, names='selected_index')

        if (len(self.accordions[category].children)-1) !=len(self.links[category]) or refresh or len(self.links[category])==0:
            vbox_children = []
            vbox_children.append(self.create_add_link(category))
            for url in self.links[category]:
                hbox = HBox(list(self.create_action_buttons(url, category)) + [Label(value=self.get_link_name(url))])
                vbox_children.append(hbox)
  
            self.accordions[category].children = [VBox(vbox_children)]
    
        self.update_accordion_title(category)
        if is_new :
            if "tabs" in self.links["ui"] and self.links["ui"]["tabs"]:
                if category in self.links["ui"]["tabs"] :
                    self.accordions[category].selected_index = 0
                else:
                    self.accordions[category].selected_index = None

        return self.accordions[category]
    
    # update the count of resources 
    def update_accordion_title(self, cat):
        # Display the update button at the end
        keep_cpt=0
        enabled_cpt=0
        for url in self.links[cat] :
            if url in self.links["ui"]["keep"] :
                keep_cpt=keep_cpt+1 
            if url in self.links["ui"]["enabled"] :
                enabled_cpt=enabled_cpt+1
        self.accordions[cat].set_title(0, str(len(self.links[cat])) +" "+ cat + " ["+str(enabled_cpt)+"/" +str(keep_cpt) +"]")
        

    #---- ui components ----# 
    # create action buttons for the link manager
    def create_action_buttons(self, url, cat):
        enabled, keep = self.get_link_status(url)
        
        toggle_enable =ToggleButton(
            layout=Layout(width='90px'),
            value=enabled, 
            description="Enable",
            tooltip="Enable this model",
            button_style=('success' if enabled else ''),
        )
    
        toggle_keep =ToggleButton(
            layout=Layout(width='90px'),
            value=keep, 
            description="Cache",
            tooltip="Cache this model into notebooks",
            button_style=('success' if keep else ''),
        )
    
        delete_button = Button(
            tooltip="Delete this model",
            button_style='danger',     
            layout=Layout(width='30px'),
            icon='trash-o' 
        )
        
        def on_toggle(change):
            self.update_link_status(url,toggle_enable.value,toggle_keep.value)
            toggle_keep.button_style = 'success' if toggle_keep.value else ''
            toggle_enable.button_style = 'success' if toggle_enable.value else ''
            self.update_accordion_title(cat)
            self.save()
        
        def delete_action(b):
            self.update_link_status(url, False, False)
            self.remove_link(cat,url)
            self.save()

        toggle_enable.observe(on_toggle, 'value')
        toggle_keep.observe(on_toggle, 'value')
        delete_button.on_click(delete_action)
        return toggle_enable , toggle_keep , delete_button

    # create add ressource ui
    def create_add_link(self, category):
        link_control = Text(
            value="",
            placeholder=category + " link...",
            layout=Layout(width='80%'),
        )
        
        add_button = Button(
            description='Add',
            layout=Layout(width='20%'),
            icon='plus' 
        )
    
        def add_action(b):
            print("adding")
            self.add_link(link_control.value,category )
            self.save()

        add_button.on_click(add_action)
        return HBox([link_control, add_button])
        
    # creating output zip/clean tools 
    def create_zip_tool(self):
        #view
        zip_button = Button(
            description='Zip',
            tooltip='Zip folder',
            icon='zipper' 
        )
        clean_button = Button(
            description='Clean',
            tooltip='Cleaning folder',
            icon='zipper'
        )
        zip_clean_button = Button(
            description='Zip & Clean',
            button_style='info', 
            tooltip='Zip folder and clean it up',
            icon='file' 
        )
    
        #actions
        def clean_outputs(b):
            !rm -rf {b}
            b.mkdir(exist_ok=True)
            
        def zip_outputs(b):
            display("Cleaning..a.")
            b.description = b.description.split('>')[0]
            is_cleaning = "clean" in b.description.lower()
            is_zipping = "zip" in b.description.lower()
            cleanup_dir = Path(output_control.value)
            if not cleanup_dir.exists():
                display("The folder doesn't exist.")
                #b.style.button_color = 'red'
                b.button_style = 'danger' 
                b.description = b.description+ '> not found...' 
                
            if any(cleanup_dir.iterdir()) :
                if is_zipping: 
                    zipped_name = f"{cleanup_dir.name}_{time.strftime('%Y-%m-%d_%H%M')}.zip" 
                    !zip -r {zipped_name} {cleanup_dir}
                    display(zipped_name+" created")
                if is_cleaning :
                    folder_name=cleanup_dir.name
                    clean_outputs(cleanup_dir)
                    display(f"{folder_name} cleaned!")
                #b.style.button_color = 'green'
                b.button_style = 'success' 
                b.description =b.description+ '> done!' 
                
            else :
                display("Empty folder.")
                #b.style.button_color = 'orange'
                b.button_style = 'warning' 
                b.description =b.description + '>empty..' 
        
        #listeners
        zip_button.on_click(zip_outputs)
        clean_button.on_click(zip_outputs)
        zip_clean_button.on_click(zip_outputs)
    
        #return ui
        return HBox([output_control, zip_button, clean_button,zip_clean_button])
                    
    # creating button to update url names
    def create_update_names_button(self ):
        update_button = Button(
            description='Update Names',        
            layout=Layout(width='200px'),
            button_style='info', 
            icon='refresh' 
        )
    
        def get_filename_url(url):
            try:
                response = requests.get(url, allow_redirects=True)
                final_url = response.url
        
                parsed_url = urlparse(final_url)
                query_params = parse_qs(parsed_url.query)
                content_disposition = query_params.get('response-content-disposition', [''])[0]
                if 'filename=' in content_disposition:
                    filename_encoded = content_disposition.split('filename=')[1].strip('"')
                    return unquote(filename_encoded)
            except Exception as e:
                print(f"Error processing URL {url}: {e}")
            return None
            
        def update_names(b):
            display("fetching names...")
            update_button.style.button_color= "orange"
            names = self.links['ui'].get('names', {})
            for category, urls in self.links.items():
                if category =="ui":
                    continue
                for url in urls:
                    if url in names and names[url]!="[error] "+url:
                        continue
                        
                    new_name=""
                    display("requesting " + url + "...")
                    if category =="extensions" :
                        new_name = url.split('/')[-1]
                        new_name=new_name.replace('.git','')
                    elif 'civitai.com' in url:
                        new_name = get_filename_url(url)
                    else:
                        new_name = url.split('/')[-1]
    
      
                    if new_name:
                        names[url] = new_name
                    else:
                        names[url] = "[error] "+url
                        display(f"No filename found for URL: {url}")
                        
                self.get_accordion(category,True)
                        
            self.links['ui']['names'] = names
            update_button.style.button_color= "green"
            self.save()
            
        update_button.on_click(update_names)
        return update_button
    
    # button to clear cached 
    def create_clear_button(self):
        cached_size=get_directory_size('/notebooks/cached')
        clear_button = Button(
            description='Clear cached '+cached_size,
            layout=Layout(width='200px'),
            button_style='info', 
            icon='recycle' 
        )
    
        def clear_cached(b):
            display("clearing files...")
            clear_button.style.button_color= "orange"
            !rm -rf '/notebooks/cached'
            clear_button.style.button_color= "green"
            clear_bin_button.description="Cleared!"
            
        clear_button.on_click(clear_cached)
        return clear_button
    
    # button to clear trash (recycle bin)
    def create_bin_button(self):
        trash_size=get_directory_size('/notebooks/.Trash*')
        clear_bin_button = Button(
            description="Clear Trash " +trash_size,
            layout=Layout(width='200px'),
            button_style='info', 
            icon='recycle' 
        )
    
        def delete_directory(pattern):
            directories = glob.glob(pattern)
            for directory in directories:
                try:
                    check_output(['rm', '-rf', directory])
                    print(f"Deleted: {directory}")
                except CalledProcessError as e:
                    print(f"Error deleting {directory}: {e}")        
        def clear_bin(b):
            trash_size=get_directory_size('/notebooks/.Trash*')
            if trash_size!="0G" :
                display("clearing files... " + trash_size)
                clear_bin_button.style.button_color= "orange"
                delete_directory('/notebooks/.Trash*') 
            clear_bin_button.style.button_color= "green"
            clear_bin_button.description="Cleared!"
            
        clear_bin_button.on_click(clear_bin)
        return clear_bin_button

    #---- views ----#
    # ressources manager
    def view_links(self):
        for category, urls in params.links.items():
            if category != "ui":
                display(HBox([Box(layout=Layout(width='18%')), self.get_accordion(category) ]))

    # output manager 
    def show_zip_tools(self):
        return self.create_zip_tool()

    # utility buttons
    def show_tools(self):
        display(HBox([Box(layout=Layout(width='18%')), self.create_update_names_button(), self.create_clear_button(), self.create_bin_button()]))


# css
style = {"description_width": "25%"}
layout = {"width": "70%"}

label_style = HTML(
    "<style> .widget-label, .widget-label-basic >  span { color: white !important; font-weight: bold;}  </style>"
)

# checkboxes
cached_config_control = Checkbox(
    value=False,  # Keep it false as default, presistent is good but I might change config in the future, also it's much more idiot-proof this way
    description="Persistent config",
    layout=layout,
    style=style,
    tooltip='save_config',
)

cached_tasks_control = Checkbox(
    value=False,  # 
    description="Persistent task scheduler",
    layout=layout,
    style=style,
    tooltip='save_tasks',
)

persistent_output_control = Checkbox(
    value=False,  # Keep it false as default, I just prefer it off by default like in old colab
    description="Save outputs to notebooks",
    layout=layout,
    style=style,
    tooltip='save_output',
)
force_cache_control = Checkbox(
    value=False,  # Keep it false as default, cuz why?!
    description="Force use of cached dependencies (faster, may break things)",
    layout=layout,
    style=style,
    tooltip='force_cached',
)
extn_update_control = Checkbox(
    value=True, # You would want newer extensions by default
    description="Update all extensions on launch",
    layout=layout,
    style=style,
    tooltip='update_extensions',
)

# textboxes
hf_token = Text(
    value="",
    description="HuggingFace token (optional)",
    placeholder="hf_xyz",
    style=style,
    layout=layout,
    tooltip='hg_token',
)
commandline_arg_control = Text(
    value="--xformers",
    description="Extra commandline arguments",
    style=style,
    layout=layout,
    tooltip='extra_cmd',
)
output_control = Text(
    value="/notebooks/outputs/",
    description="Outputs path",
    placeholder="/notebooks/outputs/",
    style= {"description_width": "46%"},
    layout=Layout(width='38%'),
    tooltip='zip_folder',
)


# params instance with all the controls to save
params=Params([output_control,cached_config_control,cached_tasks_control,persistent_output_control,force_cache_control,force_cache_control,extn_update_control,hf_token,commandline_arg_control])
params.load('links.json')

# show various tools
params.show_tools()

# textboxes
display(
    hf_token,
    commandline_arg_control,
    params.create_zip_tool(),
)

# checkboxes
display(
    cached_config_control,
    cached_tasks_control,
    persistent_output_control,
    force_cache_control,
    extn_update_control,
)

# show ressource manger
params.view_links()

### 2. Start 🚀

In [ ]:
import os
import zipfile
import time
import shutil
from subprocess import check_call
from pathlib import Path
from tempfile import NamedTemporaryFile
from urllib.parse import urlparse
from os import getenv, environ


# ---- nothing to edit in this part, just run the bloc ---- #


# Can't run without params
class StopExecution(Exception):
    def _render_traceback_(self):
        pass
try:
    params
except NameError:
    display("Please run the previous block")
    raise StopExecution


# read controllers and save parameters
params.update()
params.save()

        
# options class for easy access
class CurrentOptions:
    # fixed (Always present) command line args, needed for this to work at all
    static_args = (
        "--enable-insecure-extension-access --disable-safe-unpickle --theme dark  --listen --port 6006"
    )

    config_file = ""

    @property
    def persistent_output(self) -> bool:
        return persistent_output_control.value
        
    @property
    def cached_config(self) -> bool:
        return cached_config_control.value

    @property
    def cached_tasks(self) -> bool:
        return cached_tasks_control.value

    
    @property
    def force_cache_deps(self) -> bool:
        return force_cache_control.value

    @property
    def update_all_extensions(self) -> bool:
        return extn_update_control.value

    @property
    def optional_huggingface_token(self) -> str:
        return hf_token.value
  
    @property
    def commandline_arguments(self) -> str:
        arg_list = [self.static_args]
        if hf_token.value != "":
            arg_list.extend(["--hf-token-out", hf_token.value])
        if extn_update_control.value is True:
            arg_list.append("--update-all-extensions")
        if cached_config_control.value is True and self.config_file != "":
            arg_list.extend(["--ui-settings-file", self.config_file])
        if cached_tasks_control.value is True :
            arg_list.extend(["--agent-scheduler-sqlite-file", "/notebooks/cached/tasks.sqlite3"])
        if commandline_arg_control.value != "":
            arg_list.extend([x for x in commandline_arg_control.value.split(" ") if x != ""])
        return " ".join(arg_list)


# spawn the boi
options = CurrentOptions()

# paperspace-provided directories
notebook_dir = Path("/notebooks")
storage_dir = Path("/storage")

# local-only semi-persistent storage
content_dir = Path("/content")


    
# symlink the directory for easier access
content_dir.mkdir(exist_ok=True)
content_symlink = notebook_dir.joinpath("content")
if not all(
    (
        content_symlink.is_symlink(),
        content_symlink.resolve() == content_dir,
    )
):
    # recreate the symlink
    content_symlink.unlink(missing_ok=True)
    content_symlink.symlink_to(content_dir, target_is_directory=True)


# where we put deps
dep_cache_dir = storage_dir.joinpath("a1111_dependencies")

# where models will go
sdw_dir = content_dir.joinpath("sdw")
models_dir = sdw_dir.joinpath("models")
extensions_dir = sdw_dir.joinpath("extensions")
output_dir = notebook_dir.joinpath("outputs/") 
cached_dir = notebook_dir.joinpath("cached")

# options for aria2 to use
aria2_opts = [ "--console-log-level=error", "--summary-interval=60", "-j1", "-x1", "-s1", "-k1M", "-c"]

# download repo if needed
config_json = content_dir.joinpath("sdw/config.json")
cached_config_file = notebook_dir.joinpath("config.json")

if not config_json.exists() :
    # repo isn't here so LET'S GOOOOOOOOOO
    aria2_task = "\n".join(
        [
            "https://huggingface.co/NoCrypt/fast-repo/resolve/main/repo.tar.lz4",
            "\tout=repo.tar.lz4",
            "https://huggingface.co/NoCrypt/fast-repo/resolve/main/cache.tar.lz4",
            "\tout=cache.tar.lz4",
        ]
    )

    with NamedTemporaryFile("w") as aria2_file:
        aria2_file.write(aria2_task)
        aria2_file.flush()
        display("Downloading repo and cache. This may take a while...")
        check_call(
            ["aria2c", f"--input-file={aria2_file.name}"] + aria2_opts,
            cwd=content_dir,
        )

    display("Extracting repo...")
    check_call(["tar", "-xI", "lz4", "-f", "repo.tar.lz4", "--directory=/"], cwd=content_dir)
    content_dir.joinpath("repo.tar.lz4").unlink()

    display("Extracting cache...")
    check_call(["tar", "-xI", "lz4", "-f", "cache.tar.lz4", "--directory=/"], cwd=content_dir)
    content_dir.joinpath("cache.tar.lz4").unlink()

# deal with persistent config 
if options.cached_config :
    if not cached_config_file.exists():
        !cp {config_json} {notebook_dir}
    config_json=cached_config_file
options.config_file= str(config_json)

# deal with persistent output 
if options.persistent_output:
    if not output_dir.exists():
        output_dir.mkdir(exist_ok=True)
    !sed -i 's@"outdir_txt2img_samples": "outputs/txt2img-images"@"outdir_txt2img_samples": "{output_dir}/txt2img-images"@' {config_json}
    !sed -i 's@"outdir_img2img_samples": "outputs/img2img-images"@"outdir_img2img_samples": "{output_dir}/img2img-images"@' {config_json}
    !sed -i 's@"outdir_extras_samples": "outputs/extras-images"@"outdir_extras_samples": "{output_dir}/extras-images"@' {config_json}
    !sed -i 's@"outdir_txt2img_grids": "outputs/txt2img-grids"@"outdir_txt2img_grids": "{output_dir}/txt2img-grids"@' {config_json}
    !sed -i 's@"outdir_img2img_grids": "outputs/img2img-grids"@"outdir_img2img_grids": "{output_dir}/img2img-grids"@' {config_json}
    !sed -i 's@"outdir_save": "log/images"@"outdir_save": "{output_dir}/log/images"@' {config_json}

# set up dependency cache
if dep_cache_dir.exists() is False or options.force_cache_deps:
    if options.force_cache_deps and dep_cache_dir.exists():
        display("Forcing removal of dependency cache!")
        shutil.rmtree(dep_cache_dir)
    display("Caching dependencies. This may take a while...")
    dep_cache_dir.mkdir(exist_ok=True)
    check_call(
        ["bash", "./webui.sh", "-f", options.commandline_arguments, "--exit"],
        cwd=content_dir.joinpath("sdw"),
    )

    # recursively copy all files in /usr/local/lib/python3.10/site-packages/ to /storage/a1111_dependencies/ (except for .pyc files)
    if Path("/usr/local/lib/python3.10/site-packages/").exists():
        display("Copying dependencies to persistent cache...")
        shutil.copytree(
            "/usr/local/lib/python3.10/site-packages/",
            "/storage/a1111_dependencies/",
            symlinks=True,
            ignore=shutil.ignore_patterns("*.pyc"),
            dirs_exist_ok=True,
        )



# func to download resources
def acquire_checkpoints(checkpoint_urls: list[str], fldr: str = "Stable-diffusion", keep: bool = False):
    if len(checkpoint_urls) == 0 :
        return
    display(fldr + ": Downloading...")
    sdw_fldr=paths.get(fldr, fldr)
    dst=sdw_dir.joinpath(sdw_fldr)
    real_dst=sdw_dir.joinpath(sdw_fldr)
    if(keep) :
        if not cached_dir.exists():
            cached_dir.mkdir(exist_ok=True)
        dst=cached_dir.joinpath(fldr)
        if not dst.exists():
            dst.mkdir(exist_ok=True)
    aria2_task_lines = []
    for url in checkpoint_urls:
        filename=params.get_link_name(url)
        
        if fldr=="extensions" :
            filename = url.split('/')[-1]
            filename=filename.replace('.git','')
            if not dst.joinpath(filename).exists():
                !git clone {url} {dst}/{filename}
            else:
                !cd {dst}/{filename} && git fetch && git merge 
        if 'drive.google' in url:            
            if not dst.joinpath(filename).exists():
                if 'folders' in url:
                    !cd {dst} && gdown --folder {url} --fuzzy -c
                else:
                    !cd {dst} && gdown {url} --fuzzy -c
        elif 'civitai' in url :
            if not dst.joinpath(filename).exists():
                aria2_task_lines.extend([url,""])
        else :
            if not filename :
                filename = urlparse(url).path.split("/")[-1]
            if not dst.joinpath(filename).exists():
                aria2_task_lines.extend([url, f"\tout={filename}"])

    if len(aria2_task_lines) > 0:
        with NamedTemporaryFile("w") as aria2_file:
            aria2_file.write("\n".join(aria2_task_lines))
            aria2_file.flush()
            display(f"Downloading {len(aria2_task_lines)} checkpoints, this may take a while...")
            check_call(
                ["aria2c", f"--input-file={aria2_file.name}"] + aria2_opts,
                cwd=dst,
            )
    if keep :
        display(fldr + ": Setting up symbolic links...")
        for file in dst.iterdir():
            if fldr =="extensions":
                extension_symlink = Path(real_dst / file.name) 
                if not all(
                    (
                        extension_symlink.is_symlink(),
                        extension_symlink.resolve() == file,
                    )
                ):
                    # recreate the symlink
                    extension_symlink.unlink(missing_ok=True)
                    extension_symlink.symlink_to(file, target_is_directory=True)
                continue
            
            original_file = real_dst / file.name
            symlink_path = file
            if original_file.exists() or original_file.is_symlink():
                print(f"Removing existing file or symlink: {original_file}")
                original_file.unlink()
            try:
                original_file.symlink_to(symlink_path)
                print(f"Created symbolic link: {original_file} -> {symlink_path}")
            except FileExistsError as e:
                print(f"Failed to create symbolic link due to existing file: {e}")
        display(("[P] " if keep else "") + fldr + ": All symbolic links refreshed !")
    else:
        display(("[N] " if keep else "") + fldr +": All checkpoints already downloaded!")
       

# download, cache and link ressources      
for category, urls in params.links.items():
    if category =="ui" :
        continue

    active_urls= []
    persistent_urls= []
        
    if len(urls) >0 :
        for url in urls:
            enabled, keep = params.get_link_status(url)
            if enabled :
                if not keep :
                    if url not in active_urls:
                        active_urls.append(url)
                    if url in persistent_urls:
                        persistent_urls.remove(url)
                else :
                    if url not in persistent_urls:
                        persistent_urls.append(url)
                    if url in active_urls:
                        active_urls.remove(url)
            else:
                if url in active_urls:
                        active_urls.remove(url)
                if url in persistent_urls:
                        persistent_urls.remove(url)
    acquire_checkpoints(persistent_urls, category, True) 
    acquire_checkpoints(active_urls,category)

# start. the thing
environ["COMMANDLINE_ARGS"] = options.commandline_arguments
environ["REQS_FILE"] = "requirements_versions.txt"

display("Starting 'tensorboard'...")
display(f"Access via this URL: https://tensorboard-{getenv('PAPERSPACE_FQDN')}")

webui_user = f"""#!/bin/bash
export COMMANDLINE_ARGS='{options.commandline_arguments}'
venv_dir='-'
"""
content_dir.joinpath("sdw/webui-user.sh").write_text(webui_user)
check_call(["/usr/bin/env", "bash", "./webui.sh", "-f"], cwd=content_dir.joinpath("sdw"))


In [ ]:
# code to update webui
!cd {sdw_dir} && git pull